# 1. Preparation
## 1.1. Loading Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
from typing import Callable, List


np.set_printoptions(precision=4)

## 1.2. Numerical Simulation of Ordinary Differential Equations

Implemented the classic RK4 method and the explicit first-order Euler method.

In [2]:
def runge_kutta_4(f: Callable, x: np.ndarray, dt: float):
    '''
    Rk4 Simulator for ordinary differential equations.
    Given dx/dt = f(x) and x(t), compute x(t+dt)
    '''

    k1 = f(x)
    k2 = f(x + dt/2 * k1)
    k3 = f(x + dt/2 * k2)
    k4 = f(x + dt * k3)
    s = (k1 + 2*k2 + 2*k3 + k4)/6
    return x + s*dt


def euler(f: Callable, x: np.ndarray, dt: float):
    '''
    Euler Simulator for ordinary differential equations.
    Given dx/dt = f(x) and x(t), compute x(t+dt)
    '''

    s = f(x)
    return x + s*dt

## 1.3. Differentiable Function Class

Unifying function evaluation and differentiation into a single object makes the logic of subsequent code clearer.

In [3]:
class Differentiable:
    '''
    A class for differentiable funcitons.
    '''

    def __init__(self, f: Callable, df: Callable):
        self.__f = f
        self.__df = df

    def __call__(self, x, d=False):
        if d:
            return self.__df(x)
        else:
            return self.__f(x)

# 2. Class Implementation for Solving General Optimization Problems

Define a class that describes an optimization problem and solves it using a method similar to the Hopfield-NN approach.

In [4]:
class Minimization:

    # Penalty function
    penalty = Differentiable(lambda x: max(x, 0)**2,
                             lambda x: 2*x if x >= 0 else 0)

    def __init__(self, f: Differentiable, gs: List[Differentiable],
                 tau=1.0, penalty_rate=1.0):
        '''
        Assume x is an n-dim variable.
        Minimize `f(x)` subject to `g(x)` <= 0, for all `g` in `gs`.

        args:
        - `f`: the target function to minimize
        - `gs`: a list of constrains, where g(x) <= 0 for all g in gs.
        - `tau`: time constant controlling the speed of the dynamics.
        - `penalty_rate`: the constant controlling the magnitude of the
                          penalty function.
        '''

        self.f = f
        self.gs = gs
        self.penalty_rate = penalty_rate
        self.tau = tau

    def set_penalty_rate(self, value):
        self.penalty_rate = value

    def sum_penalty(self, x):
        return sum(self.penalty(g(x)) for g in self.gs)

    def energy(self, x):
        '''
        The energy function E(x)
        '''
        return self.f(x) + self.penalty_rate * self.sum_penalty(x)

    def dynamics(self, x):
        '''
        dx/dt where x(t) is given
        '''

        gs = np.array([g(x) for g in self.gs])
        dgdx = np.array([g(x, d=True) for g in self.gs])
        dpdg = np.array([self.penalty(g, d=True) for g in gs])

        return  -self.tau * (self.f(x, d=True) +
                             self.penalty_rate * dpdg @ dgdx)

    def iterate(self, x0, dt=1e-4, T=0.2, simulator=runge_kutta_4, pr=None):
        '''
        Simulate the dynamics of the network from `x0` and carry `nstep`
        iterations, among which the interval is set to `dt`. We can choose
        a specific (or costomed) ODE simulator, like Euler or Runge Kutta
        method.
        '''
        if pr is not None:
            self.set_penalty_rate(pr)

        x = x0
        t = 0
        k = 0
        while t < T:
            p1 = [self.penalty(g(x)) for g in self.gs][0]
            p2 = [self.penalty(g(x)) for g in self.gs][1]
            p3 = [self.penalty(g(x)) for g in self.gs][2]
            p4 = [self.penalty(g(x)) for g in self.gs][3]
            p = self.sum_penalty(x)
            e = self.energy(x)
            xnext = simulator(self.dynamics, x, dt)
            yield t, x, e, p, p1,p2,p3,p4, xnext

            k += 1
            t = k * dt
            x = xnext

# 3. Problem Solving
## 3.1 Problem Definition

In [5]:
f = Differentiable(
    lambda x: (1/30*x[0]**3 + x[1] + x[0]**2 + x[1]**2 - x[0]*x[1]),
    lambda x: np.array([0.1*x[0]**2 + 2*x[0] - x[1],
                       1 + 2*x[1] - x[0]])
)


g1 = Differentiable(
    lambda x: 2 - 5*x[0] - 2*x[1],
    lambda x: np.array([-5, -2])
)

g2 = Differentiable(
    lambda x: 1 - x[0] - 2*x[1],
    lambda x: np.array([-1,-2]
                       ))

g3 = Differentiable(
    lambda x: -x[0],
    lambda x: np.array([-1,0])
)

g4 = Differentiable(
    lambda x: -x[1],
    lambda x: np.array([0,-1])
)

m = Minimization(f, [g1, g2, g3, g4], penalty_rate=2000.0)

## 3.2 Solution



In [6]:
x0 = np.random.rand(2)
# x0 = np.array([0.4, 0.4, 0.4])

def solve(export_to: str, *args, **kargs):
    df = pd.DataFrame(
        columns=["t", "x0", "x1", "energy", "penalty", "p1", "p2", "p3", "p4"]
    )

    for k, r in enumerate(m.iterate(*args, **kargs)):
        t, x, e, p, p1, p2, p3, p4, xnext = r
        df.loc[k] = {
            "t": t,
            "x0": x[0],
            "x1": x[1],
            "energy": e,
            "penalty": p,
            "p1": p1,
            "p2": p2,
            "p3": p3,
            "p4": p4,
        }
        print(
            "Time %f, x = %s, energy %f, penalty %f, penalty1 %f, penalty2 %f, penalty3 %f, penalty4 %f"
            % (t, x, e, p, p1, p2, p3, p4)
        )

    x = xnext
    print(f"Final: x = {x}")

    df.to_excel(export_to)

In [ ]:
# solve("output1.xlsx", x0=x0, dt=1e-4, T=1.0, simulator=runge_kutta_4)
import time  # import the time module

# Record the start time
start_time = time.time()

solve("output1.xlsx", x0=x0, dt=1e-4, T=1.0, simulator=runge_kutta_4)

# Record the end time
end_time = time.time()

# Calculate the total runtime
total_time = end_time - start_time
print(f"Total runtime: {total_time:.4f} seconds")


Time 0.000000, x = [0.828  0.3518], energy 0.888803, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000100, x = [0.8279 0.3517], energy 0.888538, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000200, x = [0.8277 0.3516], energy 0.888273, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000300, x = [0.8276 0.3515], energy 0.888008, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000400, x = [0.8275 0.3515], energy 0.887743, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000500, x = [0.8273 0.3514], energy 0.887478, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.000600, x = [0.8272 0.3513], energy 0.887213, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.006300, x = [0.8194 0.3463], energy 0.872227, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006400, x = [0.8193 0.3462], energy 0.871966, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006500, x = [0.8191 0.3461], energy 0.871705, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006600, x = [0.819 0.346], energy 0.871444, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006700, x = [0.8189 0.3459], energy 0.871183, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006800, x = [0.8187 0.3459], energy 0.870922, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.006900, x = [0.8186 0.3458], energy 0.870661, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000

Time 0.013400, x = [0.8098 0.3401], energy 0.853845, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.013500, x = [0.8097 0.34  ], energy 0.853589, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.013600, x = [0.8095 0.3399], energy 0.853332, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.013700, x = [0.8094 0.3398], energy 0.853076, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.013800, x = [0.8093 0.3398], energy 0.852819, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.013900, x = [0.8091 0.3397], energy 0.852563, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.014000, x = [0.809  0.3396], energy 0.852306, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.019300, x = [0.8019 0.335 ], energy 0.838808, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019400, x = [0.8018 0.3349], energy 0.838555, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019500, x = [0.8016 0.3348], energy 0.838302, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019600, x = [0.8015 0.3347], energy 0.838049, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019700, x = [0.8014 0.3346], energy 0.837796, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019800, x = [0.8012 0.3345], energy 0.837543, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.019900, x = [0.8011 0.3345], energy 0.837290, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.025900, x = [0.7932 0.3293], energy 0.822236, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026000, x = [0.793  0.3292], energy 0.821987, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026100, x = [0.7929 0.3291], energy 0.821738, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026200, x = [0.7928 0.329 ], energy 0.821489, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026300, x = [0.7926 0.3289], energy 0.821240, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026400, x = [0.7925 0.3288], energy 0.820991, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.026500, x = [0.7924 0.3287], energy 0.820742, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

...

Time 0.104200, x = [0.6956 0.2628], energy 0.644125, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104300, x = [0.6954 0.2628], energy 0.643918, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104400, x = [0.6953 0.2627], energy 0.643710, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104500, x = [0.6952 0.2626], energy 0.643503, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104600, x = [0.6951 0.2625], energy 0.643296, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104700, x = [0.695  0.2624], energy 0.643089, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.104800, x = [0.6949 0.2623], energy 0.642882, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.110500, x = [0.6882 0.2576], energy 0.631155, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.110600, x = [0.6881 0.2575], energy 0.630951, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.110700, x = [0.6879 0.2575], energy 0.630747, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.110800, x = [0.6878 0.2574], energy 0.630542, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.110900, x = [0.6877 0.2573], energy 0.630338, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.111000, x = [0.6876 0.2572], energy 0.630134, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.111100, x = [0.6875 0.2571], energy 0.629930, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.116700, x = [0.681  0.2525], energy 0.618573, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.116800, x = [0.6809 0.2524], energy 0.618372, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.116900, x = [0.6808 0.2523], energy 0.618170, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.117000, x = [0.6806 0.2523], energy 0.617969, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.117100, x = [0.6805 0.2522], energy 0.617768, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.117200, x = [0.6804 0.2521], energy 0.617566, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.117300, x = [0.6803 0.252 ], energy 0.617365, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.123700, x = [0.6729 0.2467], energy 0.604581, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.123800, x = [0.6728 0.2467], energy 0.604382, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.123900, x = [0.6727 0.2466], energy 0.604184, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.124000, x = [0.6726 0.2465], energy 0.603986, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.124100, x = [0.6725 0.2464], energy 0.603788, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.124200, x = [0.6724 0.2463], energy 0.603590, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.124300, x = [0.6722 0.2463], energy 0.603392, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000

Time 0.129300, x = [0.6665 0.2422], energy 0.593547, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129400, x = [0.6664 0.2421], energy 0.593351, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129500, x = [0.6663 0.242 ], energy 0.593155, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129600, x = [0.6662 0.2419], energy 0.592960, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129700, x = [0.6661 0.2418], energy 0.592764, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129800, x = [0.666  0.2417], energy 0.592568, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.000000, penalty4 0.000000
Time 0.129900, x = [0.6659 0.2417], energy 0.592373, penalty 0.000000, penalty1 0.000000, penalty2 0.000000, penalty3 0.0000